In [2]:
import tensorflow as tf  # version 0.12.0rc0
import rcnn

# %config InlineBackend.figure_format = 'retina'  # double res graphs

# Load constrained dataset
from grid_dataset import load_grid_dataset
datasets = load_grid_dataset('data/grids_9x9_1000.hdf5')

# Some magic to autoreload external imports
# from http://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

- - - - - - - - - - - - - - - - - - - - 
Loaded dataset  data/grids_9x9_1000.hdf5
X.dtype =  float64  X.shape =  (1000, 9, 9)
y.dtype =  float64  y.shape =  (1000,)
- - - - - - - - - - - - - - - - - - - - 


In [3]:
# Hyperparameters
learning_rate = 1e-3
training_iters = 2000000
batch_size = 128
display_step = 50  # Steps after to give some feedback during training
validation_step = 1000  # Steps after to run a round of validation to 
                        # check against overfitting

# Network Parameters
n_input = 1000 # Data input
n_classes = 2 # Connected or not connected
grid_size = 9
height = grid_size
width = grid_size
depth = 1
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, shape=[None, height, width, depth])
    y = tf.placeholder(tf.int32, shape=[None])
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [5]:
# Construct model
rcnn = rcnn.Model(X, y, output_size=2, learning_rate=learning_rate, dropout=keep_prob)

# Initializing the variables
# Note: tensorflow needs to have this initializer after
# the model constructor above in order to catch all variables 
init = tf.initialize_all_variables()

# Merge all summaries for tensorboard writer 
merged = tf.merge_all_summaries()



In [ ]:
## Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # Profiling tools (only on tensorflow >= 0.12.0rc0)
    # Print trainable variable parameter statistics to stdout.
    # param_stats = tf.contrib.tfprof.model_analyzer.print_model_analysis(
    #    tf.get_default_graph(),
    #    tfprof_options=tf.contrib.tfprof.model_analyzer.TRAINABLE_VARS_PARAMS_STAT_OPTIONS)
    
    # param_stats is tensorflow.tfprof.TFProfNode proto. It organize the statistics
    # of each graph node in tree scructure. Let's print the root below.
    # print('total_params: %d\n' % param_stats.total_parameters)
    
    # Add a summary writer for train steps
    train_writer = tf.train.SummaryWriter("./logs", sess.graph)
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_X, batch_y = datasets.train.next_batch(batch_size)
        # Run optimization op (backprop)
        summary, _ = sess.run([merged, rcnn.train],
                              feed_dict={X: batch_X, y: batch_y,
                                         keep_prob: dropout})

        train_writer.add_summary(summary, step)
        
        # Display training steps
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            summary, loss, acc = sess.run([merged, rcnn.loss, rcnn.predict], 
                                 feed_dict={X: batch_X,
                                            y: batch_y,
                                            keep_prob: dropout})
            
            train_writer.add_summary(summary, step)
            
            print("Iteration " + str(step * batch_size) + \
                  ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc))
        
        # Run a round of validation once in a while
        if step % validation_step == 0:
            print("Validation Accuracy:", \
                  sess.run(rcnn.predict, feed_dict={X: datasets.validation.X[:batch_size],
                                                      y: datasets.validation.y[:batch_size],
                                                      keep_prob: 1.}))
            
        step += 1
    print("Optimization Finished!")
    train_writer.close()

    # Calculate accuracy
    print("Testing Accuracy:", \
          sess.run(rcnn.predict, feed_dict={X: datasets.test.X[:batch_size],
                                            y: datasets.test.y[:batch_size],
                                            keep_prob: 1.}))


Iteration 6400, Minibatch Loss= 240.916046, Training Accuracy= 0.58594
Iteration 12800, Minibatch Loss= 50.513233, Training Accuracy= 0.60938
Iteration 19200, Minibatch Loss= 14.303511, Training Accuracy= 0.57031
Iteration 25600, Minibatch Loss= 1.783519, Training Accuracy= 0.57812
Iteration 32000, Minibatch Loss= 3.793252, Training Accuracy= 0.58594
Iteration 38400, Minibatch Loss= 4.559061, Training Accuracy= 0.52344
Iteration 44800, Minibatch Loss= 1.090672, Training Accuracy= 0.57031
Iteration 51200, Minibatch Loss= 1.536652, Training Accuracy= 0.57812
Iteration 57600, Minibatch Loss= 1.305753, Training Accuracy= 0.51562
Iteration 64000, Minibatch Loss= 1.167704, Training Accuracy= 0.53906
Iteration 70400, Minibatch Loss= 1.464793, Training Accuracy= 0.60156
Iteration 76800, Minibatch Loss= 1.466116, Training Accuracy= 0.61719
Iteration 83200, Minibatch Loss= 0.914302, Training Accuracy= 0.57031
Iteration 89600, Minibatch Loss= 0.696417, Training Accuracy= 0.62500
Iteration 96000, 